In [1]:
import Rust_Machine_Learning
import plotly.express as px
import pandas as pd
from syed_ml_lib import *

In [2]:
# Get the csv data
df = pd.read_csv("../data/iris.data")
# df = pd.read_csv("../data/kmeans-dataset.csv")
# df = pd.read_csv("../data/winequality-red.csv")
# df = pd.DataFrame(data={
#   "chest_pain": [1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0],
#   "blocked_arteries": [1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0],
#   "patient_weight": [205, 180, 210, 167, 156, 125, 168, 172],
#   "heart_disease": [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0]
# })
df  

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [3]:
to_encode = [4]
encoded, feature_to_encoded_cols = one_hot_encode(df, to_encode)
encoded

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),is_ class_Iris-setosa,is_ class_Iris-versicolor,is_ class_Iris-virginica
0,5.1,3.5,1.4,0.2,1.0,0.0,0.0
1,4.9,3.0,1.4,0.2,1.0,0.0,0.0
2,4.7,3.2,1.3,0.2,1.0,0.0,0.0
3,4.6,3.1,1.5,0.2,1.0,0.0,0.0
4,5.0,3.6,1.4,0.2,1.0,0.0,0.0
...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0.0,0.0,1.0
146,6.3,2.5,5.0,1.9,0.0,0.0,1.0
147,6.5,3.0,5.2,2.0,0.0,0.0,1.0
148,6.2,3.4,5.4,2.3,0.0,0.0,1.0


In [4]:
# Prepare data for training with input rust functions expect
label_col = 6

train_data, test_data = split_train_test(encoded, 0.75, True)

# Remove any encoded data of the same class
train_data_features = train_data.copy()
remove_encoded_category_data_and_label(train_data_features, feature_to_encoded_cols, label_col)
train_data_features_list = train_data_features.values.tolist()

# Get labels
train_data_labels = train_data.iloc[:, label_col].values.tolist()

# Determine which columns in train_data_features_list are categorical
is_categorical = [False, False, False, False]

train_data_features

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
124,6.7,3.3,5.7,2.1
47,4.6,3.2,1.4,0.2
143,6.8,3.2,5.9,2.3
55,5.7,2.8,4.5,1.3
108,6.7,2.5,5.8,1.8
...,...,...,...,...
18,5.7,3.8,1.7,0.3
122,7.7,2.8,6.7,2.0
60,5.0,2.0,3.5,1.0
145,6.7,3.0,5.2,2.3


In [5]:
# Now train the model
trained_model = Rust_Machine_Learning.GradientBoost(
    train_data_features_list, is_categorical, train_data_labels, 5
)

trained_model.print()

AttributeError: module 'Rust_Machine_Learning' has no attribute 'GradientBoost'

In [ ]:
# Now let us test the model
test_data_features = test_data.copy()
remove_encoded_category_data_and_label(test_data_features, feature_to_encoded_cols, label_col)
test_data_features = test_data_features.values.tolist()
test_data_labels = test_data[test_data.columns[label_col]].values.tolist()

labelled_results = trained_model.classify(test_data_features)

num_correct = 0
for test_label, result_label in zip(test_data_labels, labelled_results):
  num_correct += test_label==result_label
print("Percent Correct: ", num_correct / len(test_data_labels))